In [3]:
!pip install dash
import plotly.express as px
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output, State

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 22.8 MB/s eta 0:00:00


In [5]:
df=pd.read_excel("/content/df_agrupado_webster_v2.xlsx")
df['year']=df['ds'].dt.year
df['month']=df['ds'].dt.month
#df=df.groupby(["Bank Name","Predicted_Pilar","year","month"])[["Nuevo_Sentiment_Score"]].mean().reset_index()
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].map('{:02}'.format), format='%Y-%m')
df['Total_Sentiment_Score'] = df.groupby(['Bank Name','date'])['Nuevo_Sentiment_Score'].transform('mean')
df.sort_values('date', inplace=True)
df

,Predicted_Pilar,ds,Bank Name,Sentiment_Score,Positive_gen,Negative_gen,Neutral_gen,Nuevo_Sentiment_Score,Sentiment_gen,year,month,date,Total_Sentiment_Score
931,Social External,2011-03-15,Fifth Third Bank,0.4181,0.3299,0.4910,0.1762,25.0000,NEGATIVE,2011,3,2011-03-01,25.000000
6036,Social Internal,2011-04-25,Dime Bank,0.9991,0.9984,0.0000,0.0014,99.8109,POSITIVE,2011,4,2011-04-01,99.810900
6038,Social Internal,2011-05-28,Dime Bank,0.4987,0.0112,0.0137,0.9750,61.5596,NEUTRAL,2011,5,2011-05-01,61.559600
6037,Social Internal,2011-05-18,Fifth Third Bank,0.9969,0.9944,0.0006,0.0049,99.3521,POSITIVE,2011,5,2011-05-01,99.352100
932,Social External,2011-07-29,Webster Bank,0.0280,0.0226,0.0785,0.0108,25.0000,MIXED,2011,7,2011-07-01,25.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,Social External,2024-04-23,First Horizon Bank,0.3291,0.3235,0.6548,0.0112,48.5016,NEGATIVE,2024,4,2024-04-01,68.913038
6033,Social External,2024-04-24,Citizens Bank,0.5484,0.0975,0.0006,0.9019,59.5339,NEUTRAL,2024,4,2024-04-01,60.338735
6034,Social External,2024-04-24,Dime Bank,0.5072,0.0146,0.0001,0.9853,61.9643,NEUTRAL,2024,4,2024-04-01,50.538808
6014,Social External,2024-04-17,Fifth Third Bank,0.0805,0.0267,0.8308,0.1075,25.0000,NEGATIVE,2024,4,2024-04-01,47.393082


In [1]:

app = Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Button("Cambiar Tema", id="theme-button", n_clicks=0),
        html.Img(src='/content/ssindex.png', style={'height': '60px', 'margin': '10px'}),
        html.H1("Monitor SSINDEX", id="title", style={'text-align': 'center'}),
    ], style={'display': 'flex', 'justify-content': 'space-between', 'align-items': 'center'}),
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': str(year), 'value': year} for year in df['year'].unique()],
                value=df['year'].max(),
                clearable=False
            ),
            dcc.Dropdown(
                id='pilar-dropdown',
                options=[{'label': pilar, 'value': pilar} for pilar in df['Predicted_Pilar'].unique()],
                value=df['Predicted_Pilar'].unique()[0],
                clearable=False
            ),
            dcc.Checklist(
                id='bank-checklist',
                options=[{'label': name, 'value': name} for name in df['Bank Name'].unique()],
                value=['Bank A'],
                inline=True
            )
        ], style={'padding': '20px', 'flex': '1'}),
        html.Div([
            dcc.Graph(id='sentiment-emoticon')
        ], style={'flex': '1', 'text-align': 'center'})
    ], style={'display': 'flex', 'justify-content': 'space-around'}),
    html.Div([
        dcc.Graph(id='yearly-pilar-line-plot', style={'flex': '1'}),
        dcc.Graph(id='total-score-line-plot', style={'flex': '1'})
    ], style={'display': 'flex', 'justify-content': 'space-around'}),
    html.Div([
        dcc.Graph(id='overall-time-line-plot', style={'flex': '1'}),
        dcc.Graph(id='sentiment-heatmap', style={'flex': '1'})
    ], style={'display': 'flex', 'justify-content': 'space-around'}),
    html.Div([
        dcc.Graph(id='bubble-chart', style={'flex': '1'}),
        dcc.Graph(id='boxplot-chart', style={'flex': '1'})
    ], style={'display': 'flex', 'justify-content': 'space-around'})
])

@app.callback(
    [Output('yearly-pilar-line-plot', 'figure'),
     Output('total-score-line-plot', 'figure'),
     Output('overall-time-line-plot', 'figure'),
     Output('main-container', 'style'),
     Output('title', 'style'),
     Output('sentiment-heatmap', 'figure'),
     Output('bubble-chart', 'figure'),
     Output('boxplot-chart', 'figure'),
     Output('sentiment-emoticon', 'figure')],
    [Input('year-dropdown', 'value'),
     Input('pilar-dropdown', 'value'),
     Input('bank-checklist', 'value'),
     Input('theme-button', 'n_clicks')],
    [State('main-container', 'style')]
)
def update_graphs(selected_year, selected_pilar, selected_banks, n_clicks, style):
    dark_theme = n_clicks % 2 != 0
    container_style = {
        'backgroundColor': '#333' if dark_theme else '#FFF',
        'padding': '20px',
        'borderRadius': '15px',
        'boxShadow': '0px 0px 10px #000'
    }
    title_style = {'text-align': 'center', 'color': 'white' if dark_theme else 'black'}

    filtered_df = df[(df['year'] == selected_year) & (df['Predicted_Pilar'] == selected_pilar) & (df['Bank Name'].isin(selected_banks))]

    fig_pilar = px.line(filtered_df, x='date', y='Nuevo_Sentiment_Score', color='Bank Name', title=f'Sentiment Score for {selected_year} - {selected_pilar}')
    fig_pilar.update_layout(
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor'],
        font_color=title_style['color']
    )

    fig_total_yearly = px.line(filtered_df, x='date', y='Total_Sentiment_Score', color='Bank Name', title=f'Total Sentiment Score for {selected_year}')
    fig_total_yearly.update_layout(
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor'],
        font_color=title_style['color']
    )

    overall_df = df[df['Bank Name'].isin(selected_banks)].groupby(['Bank Name','date']).agg({'Total_Sentiment_Score': 'mean'}).reset_index()
    fig_overall = px.line(overall_df, x='date', y='Total_Sentiment_Score', color='Bank Name', title='Overall Sentiment Score Over Time')
    fig_overall.update_layout(
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor'],
        font_color=title_style['color']
    )

    heatmap_fig = px.density_heatmap(
        filtered_df,
        x='date',
        y='Bank Name',
        z='Nuevo_Sentiment_Score',
        nbinsx=30,
        nbinsy=10,
        color_continuous_scale='Viridis'
    )
    heatmap_fig.update_layout(
        title={
            'text': 'Heatmap of Sentiment Scores',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'color': title_style['color']}
        },
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor']
    )

    bubble_fig = px.scatter(filtered_df, x='date', y='Nuevo_Sentiment_Score', size='Total_Sentiment_Score', color='Bank Name', title='Bubble Chart of Sentiments')
    bubble_fig.update_layout(
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor'],
        font_color=title_style['color']
    )

    boxplot_fig = px.box(filtered_df, x='Bank Name', y='Nuevo_Sentiment_Score', color='Bank Name', title='Boxplot of Sentiment Scores by Bank')
    boxplot_fig.update_layout(
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor'],
        font_color=title_style['color']
    )

    # Emoticono basado en Sentiment_gen
    last_sentiment = filtered_df['Sentiment_gen'].iloc[-1]
    emoticon_url = {
        'POSITIVE': 'https://path/to/happy_emoticon.png',
        'NEUTRAL': 'https://path/to/neutral_emoticon.png',
        'NEGATIVE': 'https://path/to/sad_emoticon.png'
    }.get(last_sentiment, 'https://path/to/neutral_emoticon.png')  # Default to neutral

    emoticon_fig = {
        'data': [],
        'layout': {
            'images': [
                {
                    'source': emoticon_url,
                    'xref': 'paper',
                    'yref': 'paper',
                    'x': 0.5,
                    'y': 0.5,
                    'sizex': 0.5,
                    'sizey': 0.5,
                    'xanchor': 'center',
                    'yanchor': 'middle'
                }
            ],
            'xaxis': {'visible': False},
            'yaxis': {'visible': False},
            'paper_bgcolor': container_style['backgroundColor'],
            'plot_bgcolor': container_style['backgroundColor']
        }
    }

    return fig_pilar, fig_total_yearly, fig_overall, container_style, title_style, heatmap_fig, bubble_fig, boxplot_fig, emoticon_fig

# Ejecución de la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


NameError: name 'Dash' is not defined

In [ ]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output, State
import plotly.express as px

# Inicialización de la aplicación Dash
app = Dash(__name__)

# Layout de la aplicación
app.layout = html.Div([
    html.Div([
        html.Button("Cambiar Tema", id="theme-button", n_clicks=0),
        html.Img(src='/content/ssindex.png', style={'height': '60px', 'margin': '10px'}),
        html.H1("Monitor SSINDEX", id="title", style={'text-align': 'center'}),
        html.Div([
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': str(year), 'value': year} for year in df['year'].unique()],
                value=df['year'].max(),
                clearable=False
            ),
            dcc.Dropdown(
                id='pilar-dropdown',
                options=[{'label': pilar, 'value': pilar} for pilar in df['Predicted_Pilar'].unique()],
                value=df['Predicted_Pilar'].unique()[0],
                clearable=False
            ),
            dcc.Checklist(
                id='bank-checklist',
                options=[{'label': name, 'value': name} for name in df['Bank Name'].unique()],
                value=['Bank A'],
                inline=True
            )
        ], style={'padding': '20px'}),
        dcc.Graph(id='yearly-pilar-line-plot'),
        dcc.Graph(id='total-score-line-plot'),
        dcc.Graph(id='overall-time-line-plot'),
        dcc.Graph(id='sentiment-heatmap'),
        dcc.Graph(id='bubble-chart'),
        dcc.Graph(id='boxplot-chart'),
    ], id='main-container')
])

# Callback para actualizar los gráficos y manejar el tema
@app.callback(
    [Output('yearly-pilar-line-plot', 'figure'),
     Output('total-score-line-plot', 'figure'),
     Output('overall-time-line-plot', 'figure'),
     Output('main-container', 'style'),
     Output('title', 'style'),
     Output('sentiment-heatmap', 'figure'),
     Output('bubble-chart', 'figure'),
     Output('boxplot-chart', 'figure')],
    [Input('year-dropdown', 'value'),
     Input('pilar-dropdown', 'value'),
     Input('bank-checklist', 'value'),
     Input('theme-button', 'n_clicks')],
    [State('main-container', 'style')]
)
def update_graphs(selected_year, selected_pilar, selected_banks, n_clicks, style):
    dark_theme = n_clicks % 2 != 0
    container_style = {
        'backgroundColor': '#333' if dark_theme else '#FFF',
        'padding': '20px',
        'borderRadius': '15px',
        'boxShadow': '0px 0px 10px #000'
    }
    title_style = {'text-align': 'center', 'color': 'white' if dark_theme else 'black'}

    # Filtrado de datos según las entradas del usuario
    filtered_df = df[(df['year'] == selected_year) & (df['Predicted_Pilar'] == selected_pilar) & (df['Bank Name'].isin(selected_banks))]

    # Gráficos actualizados
    fig_pilar = px.line(filtered_df, x='date', y='Nuevo_Sentiment_Score', color='Bank Name', title=f'Sentiment Score for {selected_year} - {selected_pilar}')
    fig_pilar.update_layout(
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor'],
        font_color=title_style['color']
    )

    fig_total_yearly = px.line(filtered_df, x='date', y='Total_Sentiment_Score', color='Bank Name', title=f'Total Sentiment Score for {selected_year}')
    fig_total_yearly.update_layout(
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor'],
        font_color=title_style['color']
    )

    overall_df = df[df['Bank Name'].isin(selected_banks)].groupby(['Bank Name','date']).agg({'Total_Sentiment_Score': 'mean'}).reset_index()
    fig_overall = px.line(overall_df, x='date', y='Total_Sentiment_Score', color='Bank Name', title='Overall Sentiment Score Over Time')
    fig_overall.update_layout(
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor'],
        font_color=title_style['color']
    )

    heatmap_fig = px.density_heatmap(
        filtered_df,
        x='date',
        y='Bank Name',
        z='Nuevo_Sentiment_Score',
        nbinsx=30,
        nbinsy=10,
        color_continuous_scale='Viridis'
    )
    heatmap_fig.update_layout(
        title={
            'text': 'Heatmap of Sentiment Scores',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'color': title_style['color']}
        },
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor']
    )

    bubble_fig = px.scatter(filtered_df, x='date', y='Nuevo_Sentiment_Score', size='Total_Sentiment_Score', color='Bank Name', title='Bubble Chart of Sentiments')
    bubble_fig.update_layout(
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor'],
        font_color=title_style['color']
    )

    boxplot_fig = px.box(filtered_df, x='Bank Name', y='Nuevo_Sentiment_Score', color='Bank Name', title='Boxplot of Sentiment Scores by Bank')
    boxplot_fig.update_layout(
        plot_bgcolor=container_style['backgroundColor'],
        paper_bgcolor=container_style['backgroundColor'],
        font_color=title_style['color']
    )

    return fig_pilar, fig_total_yearly, fig_overall, container_style, title_style, heatmap_fig, bubble_fig, boxplot_fig

# Ejecución de la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output, State
import plotly.express as px

# Inicialización de la aplicación Dash
app = Dash(__name__)

# Layout de la aplicación
app.layout = html.Div([
    html.Div([
        html.Button("Cambiar Tema", id="theme-button", n_clicks=0),
        html.Img(src='/content/ssindex.png', style={'height': '60px', 'margin': '10px'}),
        html.H1("Monitor SSINDEX", id="title", style={'text-align': 'center'}),
        html.Div([
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': str(year), 'value': year} for year in df['year'].unique()],
                value=df['year'].max(),
                clearable=False
            ),
            dcc.Dropdown(
                id='pilar-dropdown',
                options=[{'label': pilar, 'value': pilar} for pilar in df['Predicted_Pilar'].unique()],
                value=df['Predicted_Pilar'].unique()[0],
                clearable=False
            ),
            dcc.Checklist(
                id='bank-checklist',
                options=[{'label': name, 'value': name} for name in df['Bank Name'].unique()],
                value=['Webster Bank'],
                inline=True
            )
        ], style={'padding': '20px'}),
        dcc.Graph(id='yearly-pilar-line-plot'),
        dcc.Graph(id='total-score-line-plot'),
        dcc.Graph(id='overall-time-line-plot'),
        dcc.Graph(id='sentiment-heatmap'),
        dcc.Graph(id='bubble-chart'),
        dcc.Graph(id='boxplot-chart'),
    ], id='main-container')
])

# Callback para actualizar los gráficos y manejar el tema
@app.callback(
    [Output('yearly-pilar-line-plot', 'figure'),
     Output('total-score-line-plot', 'figure'),
     Output('overall-time-line-plot', 'figure'),
     Output('main-container', 'style'),
     Output('title', 'style'),
     Output('sentiment-heatmap', 'figure'),
    Output('bubble-chart', 'figure'),
    Output('boxplot-chart', 'figure')],
    [Input('year-dropdown', 'value'),
     Input('pilar-dropdown', 'value'),
     Input('bank-checklist', 'value'),
     Input('theme-button', 'n_clicks')],
    [State('main-container', 'style')]
)
def update_graphs(selected_year, selected_pilar, selected_banks, n_clicks, style):
    dark_theme = n_clicks % 2 != 0
    container_style = {
        'backgroundColor': '#333' if dark_theme else '#FFF',
        'padding': '20px',
        'borderRadius': '15px',
        'boxShadow': '0px 0px 10px #000'
    }
    title_style = {'text-align': 'center', 'color': 'white' if dark_theme else 'black'}

    # Filtrado de datos según las entradas del usuario
    filtered_df = df[(df['year'] == selected_year) & (df['Predicted_Pilar'] == selected_pilar) & (df['Bank Name'].isin(selected_banks))]

    # Gráficos actualizados
    fig_pilar = px.line(filtered_df, x='date', y='Nuevo_Sentiment_Score', color='Bank Name', title=f'Sentiment Score for {selected_year} - {selected_pilar}')
    fig_total_yearly = px.line(filtered_df, x='date', y='Total_Sentiment_Score', color='Bank Name',title=f'Total Sentiment Score for {selected_year}')
    overall_df = df[df['Bank Name'].isin(selected_banks)].groupby(['Bank Name','date']).agg({'Total_Sentiment_Score': 'mean'}).reset_index()
    fig_overall = px.line(overall_df, x='date', y='Total_Sentiment_Score', color='Bank Name',title='Overall Sentiment Score Over Time')
    heatmap_fig = px.density_heatmap(
        filtered_df,
        x='date',
        y='Bank Name',
        z='Nuevo_Sentiment_Score',
        nbinsx=30,  # Puedes ajustar esto según la densidad deseada
        nbinsy=10,
        color_continuous_scale='Viridis'  # Escala de color para mejor visualización
    )
    heatmap_fig.update_layout(
        title={
            'text': 'Heatmap of Sentiment Scores',
            'x': 0.5,
            'xanchor': 'center',
        })
    bubble_fig = px.scatter(filtered_df, x='date', y='Nuevo_Sentiment_Score', size='Total_Sentiment_Score', color='Bank Name', title='Bubble Chart of Sentiments')
    boxplot_fig = px.box(filtered_df, x='Bank Name', y='Nuevo_Sentiment_Score', color='Bank Name', title='Boxplot of Sentiment Scores by Bank')
    return fig_pilar, fig_total_yearly, fig_overall, heatmap_fig,bubble_fig,boxplot_fig,  container_style, title_style

# Ejecución de la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')


<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px

# Suponiendo que df_aux es tu DataFrame ya cargado y preparado
df = df_aux
# Asegurémonos de que la columna 'month' es reconocida en formato de mes completo para el datetime
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].map('{:02}'.format), format='%Y-%m')  # Ajustamos el formato para incluir dos dígitos en el mes
df['Total_Sentiment_Score'] = df.groupby('date')['Nuevo_Sentiment_Score'].transform('mean')  # Calculamos la media del score por cada fecha
df.sort_values('date', inplace=True)  # Ordenamos por fecha para asegurar la secuencia cronológica

# Inicialización de la aplicación Dash
app = Dash(__name__)

# Layout de la aplicación
app.layout = html.Div([
    html.Div([
        html.Img(src='/content/ssindex.png', style={'height': '60px', 'margin': '10px'}),
        html.H1("Monitor SSINDEX", style={'font-family': 'Lato', 'color': 'white', 'text-align': 'center'}),
        html.Div([
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': str(year), 'value': year} for year in df['year'].unique()],
                value=df['year'].min(),
                clearable=False
            ),
            dcc.Dropdown(
                id='pilar-dropdown',
                options=[{'label': pilar, 'value': pilar} for pilar in df['Predicted_Pilar'].unique()],
                value=df['Predicted_Pilar'].unique()[0],
                clearable=False
            ),
             dcc.Checklist(
                id='bank-checklist',
                options=[{'label': name, 'value': name} for name in df['Bank Name'].unique()],
                value=['Webster Bank'],
                inline=True
            )
        ], style={'padding': '20px', 'width': '300px', 'display': 'inline-block', 'verticalAlign': 'top'}),
        dcc.Graph(id='yearly-pilar-line-plot'),
        dcc.Graph(id='total-score-line-plot'),
        dcc.Graph(id='overall-time-line-plot')
    ], style={'backgroundColor': '#333', 'padding': '20px', 'borderRadius': '15px', 'boxShadow': '0px 0px 10px #000'})
])

# Callback para actualizar los gráficos basados en el año y pilar seleccionados
@app.callback(
    [Output('yearly-pilar-line-plot', 'figure'),
     Output('total-score-line-plot', 'figure'),
     Output('overall-time-line-plot', 'figure')],
    [Input('year-dropdown', 'value'),
     Input('pilar-dropdown', 'value')]
)
def update_graphs(selected_year, selected_pilar):
    # Gráfico de línea específico por pilar y año
    filtered_df = df[(df['year'] == selected_year) & (df['Predicted_Pilar'] == selected_pilar)]
    fig_pilar = px.line(
        filtered_df,
        x='date',
        y='Nuevo_Sentiment_Score',
        color='Bank Name',
        title=f'Sentiment Score for {selected_year} - {selected_pilar}',
        markers=True
    )

    # Gráfico de línea de puntuación total por año seleccionado
    total_yearly_df = df[df['year'] == selected_year]
    fig_total_yearly = px.line(
        total_yearly_df,
        x='date',
        y='Total_Sentiment_Score',
        title=f'Total Sentiment Score for {selected_year}',
        markers=True
    )

    # Gráfico de línea para toda la historia de datos
    overall_df = df.groupby('date').agg({'Total_Sentiment_Score': 'mean'}).reset_index()
    fig_overall = px.line(
        overall_df,
        x='date',
        y='Total_Sentiment_Score',
        title='Overall Sentiment Score Over Time',
        markers=True
    )

    # Actualización del layout para todos los gráficos
    for fig in (fig_pilar, fig_total_yearly, fig_overall):
        fig.update_layout(
            plot_bgcolor='#333',
            paper_bgcolor='#333',
            font=dict(
                family="Lato, Arial",
                size=18,
                color='white'
            )
        )
    return fig_pilar, fig_total_yearly, fig_overall

# Función para ejecutar la aplicación dentro de Jupyter
def run_dash_app():
    app.run_server(mode='inline', port=8050)

# Ejecución de la aplicación
run_dash_app()


<IPython.core.display.Javascript object>

In [ ]:
df = df_aux
df['month'] = pd.to_datetime(df['month'], format='%m').dt.strftime('%B')  # Convert month numbers to full names
df['Total_Sentiment_Score'] = df.groupby(['year', 'month'])['Nuevo_Sentiment_Score'].transform('mean')  # Total score per month
df.sort_values(by=['year', 'month'], inplace=True)  # Ensure correct chronological order

app = Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Img(src='/content/ssindex.png', style={'height': '60px', 'margin': '10px'}),
        html.H1("Monitor SSINDEX", style={'font-family': 'Lato', 'color': 'white', 'text-align': 'center'}),
        html.Div([
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': year, 'value': year} for year in df['year'].unique()],
                value=df['year'].min(),
                clearable=False
            ),
            dcc.Dropdown(
                id='pilar-dropdown',
                options=[{'label': pilar, 'value': pilar} for pilar in df['Predicted_Pilar'].unique()],
                value=df['Predicted_Pilar'].unique()[0],
                clearable=False
            )
        ], style={'padding': '20px', 'width': '300px', 'display': 'inline-block', 'verticalAlign': 'top'}),
        dcc.Graph(id='yearly-pilar-line-plot'),
        dcc.Graph(id='total-score-line-plot'),
        dcc.Graph(id='overall-time-line-plot')
    ], style={'backgroundColor': '#333', 'padding': '20px', 'borderRadius': '15px', 'boxShadow': '0px 0px 10px #000'})
])

# Callback to update the graphs based on selected year and pilar
@app.callback(
    [Output('yearly-pilar-line-plot', 'figure'),
     Output('total-score-line-plot', 'figure'),
     Output('overall-time-line-plot', 'figure')],
    [Input('year-dropdown', 'value'),
     Input('pilar-dropdown', 'value')]
)
def update_graphs(selected_year, selected_pilar):
    # Yearly Pilar-specific Line Plot
    filtered_df = df[(df['year'] == selected_year) & (df['Predicted_Pilar'] == selected_pilar)]
    fig_pilar = px.line(
        filtered_df,
        x='month',
        y='Nuevo_Sentiment_Score',
        color='Bank Name',
        title=f'Sentiment Score for {selected_year} - {selected_pilar}',
        labels={'month': 'Month', 'Nuevo_Sentiment_Score': 'Sentiment Score'},
        markers=True
    )

    # Total Score by Year Line Plot
    total_yearly_df = df[df['year'] == selected_year]
    fig_total_yearly = px.line(
        total_yearly_df,
        x='month',
        y='Total_Sentiment_Score',
        title=f'Total Sentiment Score for {selected_year}',
        labels={'month': 'Month', 'Total_Sentiment_Score': 'Total Sentiment Score'},
        markers=True
    )

    # Overall Time Line Plot for all data
    overall_df = df.groupby(['year', 'month']).agg({'Total_Sentiment_Score': 'sum'}).reset_index()
    fig_overall = px.line(
        overall_df,
        x='month',
        y='Total_Sentiment_Score',
        title='Overall Sentiment Score Over Time',
        labels={'month': 'Month', 'Total_Sentiment_Score': 'Total Sentiment Score'},
        markers=True
    )

    # Update the layout for all figures to use a more professional font and add dark theme
    for fig in (fig_pilar, fig_total_yearly, fig_overall):
        fig.update_layout(
            plot_bgcolor='#333',
            paper_bgcolor='#333',
            font=dict(
                family="Lato, Arial",
                size=18,
                color='white'
            )
        )
    return fig_pilar, fig_total_yearly, fig_overall

# Function to run the app within Jupyter
def run_dash_app():
    app.run_server(mode='inline', port=8050)

# Call the function to run the app
run_dash_app()



<IPython.core.display.Javascript object>

In [ ]:
df

,Bank Name,Predicted_Pilar,year,month,Nuevo_Sentiment_Score,Total_Sentiment_Score
56,Webster Bank,Social External,2011,July,25.000000,25.000000
57,Webster Bank,Social External,2011,October,25.000000,25.000000
60,Webster Bank,Social External,2012,December,25.000000,25.000000
58,Webster Bank,Social External,2012,July,96.717900,96.717900
59,Webster Bank,Social External,2012,October,74.725033,74.725033
...,...,...,...,...,...,...
167,Webster Bank,Social External,2024,January,49.098162,53.139656
251,Webster Bank,Social Internal,2024,January,57.181150,53.139656
42,Webster Bank,Environmental,2024,March,68.220900,57.098967
169,Webster Bank,Social External,2024,March,31.247700,57.098967


In [ ]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px

# Supongamos que df_aux es tu DataFrame ya cargado y preparado
df = df_aux
#df['date'] = pd.to_datetime(df['year'].astype(str) + df['month'], format='%Y%B')  # Combinamos año y mes en una columna datetime
#df['Total_Sentiment_Score'] = df.groupby(['date'])['Nuevo_Sentiment_Score'].transform('mean')  # Media del score por mes
#df.sort_values(by='date', inplace=True)  # Ordenamos por la nueva columna date

# Inicialización de la aplicación Dash
app = Dash(__name__)

# Layout de la aplicación
app.layout = html.Div([
    html.Div([
        html.Img(src='/content/ssindex.png', style={'height': '60px', 'margin': '10px'}),
        html.H1("Monitor SSINDEX", style={'font-family': 'Lato', 'color': 'white', 'text-align': 'center'}),
        html.Div([
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': year, 'value': year} for year in df['year'].unique()],
                value=df['year'].min(),
                clearable=False
            ),
            dcc.Dropdown(
                id='pilar-dropdown',
                options=[{'label': pilar, 'value': pilar} for pilar in df['Predicted_Pilar'].unique()],
                value=df['Predicted_Pilar'].unique()[0],
                clearable=False
            )
        ], style={'padding': '20px', 'width': '300px', 'display': 'inline-block', 'verticalAlign': 'top'}),
        dcc.Graph(id='yearly-pilar-line-plot'),
        dcc.Graph(id='total-score-line-plot'),
        dcc.Graph(id='overall-time-line-plot')
    ], style={'backgroundColor': '#333', 'padding': '20px', 'borderRadius': '15px', 'boxShadow': '0px 0px 10px #000'})
])

# Callback para actualizar los gráficos basados en el año y pilar seleccionados
@app.callback(
    [Output('yearly-pilar-line-plot', 'figure'),
     Output('total-score-line-plot', 'figure'),
     Output('overall-time-line-plot', 'figure')],
    [Input('year-dropdown', 'value'),
     Input('pilar-dropdown', 'value')]
)
def update_graphs(selected_year, selected_pilar):
    # Gráfico de línea específico por pilar y año
    filtered_df = df[(df['year'] == selected_year) & (df['Predicted_Pilar'] == selected_pilar)]
    fig_pilar = px.line(
        filtered_df,
        x='month',
        y='Nuevo_Sentiment_Score',
        color='Bank Name',
        title=f'Sentiment Score for {selected_year} - {selected_pilar}',
        labels={'month': 'Month', 'Nuevo_Sentiment_Score': 'Sentiment Score'},
        markers=True
    )

    # Gráfico de línea de puntuación total por año seleccionado
    total_yearly_df = df[df['year'] == selected_year]
    fig_total_yearly = px.line(
        total_yearly_df,
        x='month',
        y='Total_Sentiment_Score',
        title=f'Total Sentiment Score for {selected_year}',
        labels={'month': 'Month', 'Total_Sentiment_Score': 'Total Sentiment Score'},
        markers=True
    )

    # Gráfico de línea para toda la historia de datos
    overall_df = df.groupby('date').agg({'Total_Sentiment_Score': 'mean'}).reset_index()
    fig_overall = px.line(
        overall_df,
        x='date',
        y='Total_Sentiment_Score',
        title='Overall Sentiment Score Over Time',
        labels={'date': 'Date', 'Total_Sentiment_Score': 'Total Sentiment Score'},
        markers=True
    )

    # Actualización del layout para todos los gráficos
    for fig in (fig_pilar, fig_total_yearly, fig_overall):
        fig.update_layout(
            plot_bgcolor='#333',
            paper_bgcolor='#333',
            font=dict(
                family="Lato, Arial",
                size=18,
                color='white'
            )
        )
    return fig_pilar, fig_total_yearly, fig_overall

# Función para ejecutar la aplicación dentro de Jupyter
def run_dash_app():
    app.run_server(mode='inline', port=8050)

# Ejecución de la aplicación
run_dash_app()


<IPython.core.display.Javascript object>